# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import friedmanchisquare, ttest_rel
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm  import SVC
from sklearn.ensemble  import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from itertools import combinations

# Análise e tratamento dos dados

In [ ]:
df = pd.read_csv('winequality-red.csv')
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.plot(kind='box', subplots=True, layout=(3, 4), figsize=(16, 10), sharex=False, sharey=False)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(11,9))
plotnumber=1

cont_col = [col for col in df.columns if col != 'quality']

for column in cont_col:
    ax=plt.subplot(4,3,plotnumber)
    sns.barplot(x='quality', y=df[column], data=df, palette = 'viridis', errorbar=None, hue='quality', legend=False)
    
    plt.xlabel('Quality')
    plt.ylabel(column)
    plotnumber+=1
plt.tight_layout()

In [ ]:
correlations = df.corr(numeric_only=True)['quality'].drop('quality').sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=correlations.values, y=correlations.index, palette='viridis', hue=correlations.index, legend=False)
plt.xlabel('Correlação com a qualidade')
plt.ylabel('Atributo')
plt.title('Influência dos atributos na qualidade do vinho')
plt.tight_layout()
plt.show()

In [ ]:
crr = df.corr()
plt.figure(figsize=(9, 6))
sns.heatmap(crr, annot=True, fmt='.2f', cmap='coolwarm', square=True, cbar_kws={"shrink": .8}, linewidths=0.5)
plt.title('Matriz de Correlação')
plt.show()

In [ ]:
df.quality.unique()

In [ ]:
df = df.replace({'quality' : {
        8 : 'Good',
        7 : 'Good',
        6 : 'Middle',
        5 : 'Middle',
        4 : 'Bad',
        3 : 'Bad',
        }}
)
df.head()

In [ ]:
X_temp = df.drop(columns='quality')
y = df.quality
y

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1)).fit_transform(X_temp)
x = pd.DataFrame(scaler, columns=X_temp.columns)
x.describe()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)

# Modelos

In [ ]:
best_estimators = {}
cv_distributions = {}

## LogisticRegression

In [ ]:
parameters = {
    'C' : [0.001, 0.01, 0.1, 1.0, 10, 100, 1000],
    'class_weight' : ['balanced'],
    'solver' : ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky']
}


lr = LogisticRegression()
lr_cv = GridSearchCV(estimator=lr, param_grid=parameters, cv=20).fit(x_train, y_train)

print('Tuned hyper parameters : ', lr_cv.best_params_)
print('accuracy : ', lr_cv.best_score_)

best_lr = lr_cv.best_estimator_
best_estimators['LogisticRegression'] = best_lr

lr_score = best_lr.score(x_test, y_test)
print('LogisticRegression score : ', lr_score)

## RandomForestClassifier

In [ ]:
parameters = {
    'n_estimators': [50, 150, 500],
    'criterion':    ['gini','entropy','log_loss'],
    'max_features': ['sqrt','log2']
}


rf = RandomForestClassifier(n_jobs=-1)
rf_cv = GridSearchCV(estimator=rf, param_grid=parameters, cv=10).fit(x_train, y_train)

print('Tuned hyper parameters : ', rf_cv.best_params_)
print('accuracy : ', rf_cv.best_score_)

best_rf = rf_cv.best_estimator_
best_estimators['RandomForest'] = best_rf

rf_score = best_rf.score(x_test, y_test)
print('RandomForest score : ', rf_score)

## SVM (SVC)

In [ ]:
parameters = {
    'C' : [0.001, 0.01, 0.1, 1.0, 10, 100],
    'gamma' : [0.001, 0.01, 0.1, 1.0, 10, 100],
    'class_weight' : ['balanced'],
}

svc = SVC()
svc_cv = GridSearchCV(estimator=svc, param_grid=parameters, cv=5).fit(x_train, y_train)

print('Tuned hyper parameters : ', svc_cv.best_params_)
print('accuracy : ', svc_cv.best_score_)

best_svc = svc_cv.best_estimator_
best_estimators['SVC'] = best_svc

svc_score = best_svc.score(x_test, y_test)
print('SVC score : ', svc_score)

## DecisionTreeClassifier

In [ ]:
parameters = {
    'criterion': ['gini', 'entropy'],
    'splitter' : ['best', 'random'],
    'max_depth': list(np.arange(4, 30, 1)),
    'min_samples_split':[2, 5]
}


dt = DecisionTreeClassifier()
dt_cv = GridSearchCV(estimator=dt, param_grid=parameters, cv=20).fit(x_train, y_train)

print('Tuned hyper parameters : ', dt_cv.best_params_)
print('accuracy : ', dt_cv.best_score_)

best_dt = dt_cv.best_estimator_
best_estimators['DecisionTreeClassifier'] = best_dt

dt_score = best_dt.score(x_test, y_test)
print('DecisionTreeClassifier score : ', dt_score)

## KNeighborsClassifier

In [ ]:
parameters = {
    'n_neighbors' : list(np.arange(3, 50, 2)),
    'weights': ['uniform', 'distance'],
    'p' : [1, 2, 3, 4]
}


knn = KNeighborsClassifier()
knn_cv = GridSearchCV(estimator=knn, param_grid=parameters, cv=20).fit(x_train, y_train)

print('Tuned hyper parameters : ', knn_cv.best_params_)
print('accuracy : ', knn_cv.best_score_)

best_knn = knn_cv.best_estimator_
best_estimators['KNeighborsClassifier'] = best_knn

knn_score = best_knn.score(x_test, y_test)
print('KNeighborsClassifier score : ', knn_score)

## Rede Neural

In [ ]:
parameters = {
    'hidden_layer_sizes': [(50,), (100,)],
    'activation':  ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001] 
}


mlp = MLPClassifier(max_iter=500)
mlp_cv = GridSearchCV(estimator=mlp, param_grid=parameters, cv=10).fit(x_train, y_train)

print('Tuned hyper parameters : ', mlp_cv.best_params_)
print('accuracy : ', mlp_cv.best_score_)

best_mlp = mlp_cv.best_estimator_
best_estimators['MLPClassifier'] = best_mlp

mlp_score = best_mlp.score(x_test, y_test)
print('MLPClassifier score : ', mlp_score)

# Análise de resultados

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=30, random_state=42)

In [ ]:
for name, model in best_estimators.items():
    print(f"\n=== Métricas no hold-out para {name} ===")

    y_pred = model.predict(x_test)
    print("-- Classification Report --")
    print(classification_report(
        y_test, y_pred, digits=3,
        zero_division=0
    ))
    print("-- Confusion Matrix --")
    print(confusion_matrix(y_test, y_pred))


In [ ]:
for name, est in best_estimators.items():
    scores = cross_val_score(
        est, x, y,
        cv=cv,
        scoring='accuracy',
        n_jobs=-1
    )
    cv_distributions[name] = scores
    print(f"{name}: mean={scores.mean():.4f} ± {scores.std():.4f}")

In [ ]:
df = pd.DataFrame(cv_distributions)
sns.boxplot(data=df)

In [ ]:
stat, p = friedmanchisquare(*cv_distributions.values())
print(f"Friedman χ²={stat:.3f}, p={p:.4f}")
if p < 0.05:
    print("→ Há diferença significativa em pelo menos um par de algoritmos")

In [ ]:
alpha = 0.05
m = len(cv_distributions)
comparisons = list(combinations(cv_distributions.keys(), 2))
for a, b in comparisons:
    s1 = cv_distributions[a]
    s2 = cv_distributions[b]
    stat, pval = ttest_rel(s1, s2)
    p_adj = min(pval * len(comparisons), 1.0)
    print(f"{a} vs {b}: t={stat:.3f}, p={pval:.4f}, p_adj={p_adj:.4f}",
        "→ dif. sign." if p_adj<alpha else "→ não dif.")

In [ ]:
cv_means = {}
for name, value in cv_distributions.items():
    cv_means[name] = value.mean()

In [ ]:
plt.figure(figsize=(10, 6))
sorted_items = sorted(cv_means.items(), key=lambda item: item[1])
sns.barplot(x=[k for k, v in sorted_items], y=[v for k, v in sorted_items], palette='viridis', hue=[v for k, v in sorted_items], legend=False)
plt.xlabel('Modelos')
plt.ylabel('Mean Score')
plt.title('Comparação de Modelos de Classificação')
plt.tight_layout()
plt.show()

In [ ]:
# === 1. Resumo estatístico dos modelos ===
import numpy as np
import pandas as pd
from scipy.stats import sem, t

# Função para calcular IC95%
def ci95(scores, confidence=0.95):
    m    = scores.mean()
    se   = sem(scores)
    h    = se * t.ppf((1 + confidence) / 2, len(scores) - 1)
    return m, m-h, m+h

# Monta a lista de resumos
rows = []
for name, scores in cv_distributions.items():
    m, lo, hi = ci95(scores)
    held = best_estimators[name].score(x_test, y_test)
    rows.append({
        'modelo':     name,
        'mean_cv':    m,
        'std_cv':     scores.std(),
        'ci95_lo':    lo,
        'ci95_hi':    hi,
        'heldout':    held
    })

# Cria o DataFrame e exibe
df_summary = pd.DataFrame(rows).set_index('modelo')
df_summary
